In [2]:
import re
from nvbot_utils.utils.caching import Cache


In [3]:
from configs.settings import get_settings

redis_secret = get_settings().REDIS_SECRETS
cache = Cache(redis_secrets= redis_secret)

def convert_to_snake_case(text):
    # Step 1: Add an underscore before each capital letter if it is preceded by a lowercase letter or a number
    text = re.sub(r'(?<!\s)(?<=\w)([A-Z])', r'_\1', text).lower()
    # Step 2: Replace all whitespace characters with underscores
    text = re.sub(r'\s+', '_', text)
    # Step 3: Remove any leading underscores (if the original string started with a capital letter)
    text = re.sub(r'^_', '', text)
    # Step 4: Remove all non-alphanumeric characters except underscores
    text = re.sub(r'[^a-z0-9_]', '', text)
    return text


# query = "when_are_the_next_free_days"
query_key_hash = 'NVINFO_RESPONSE_CACHE'
# query_key = f'NVINFO_QUERY_CACHE_KEY:{query}'
query_key = "NVINFO_QUERY_CACHE_KEY:when_is_the_ntech_conference"
def check_response_cache_exists(query_key):
    if query_key:
        print (query_key)
        response = cache.get_from_cache(query_key, query_key_hash)
        if response and (response not in ["None", "null"]):
            return True, response
        else:
            return False, ""
        

found, response = check_response_cache_exists(query_key)
response

NVINFO_QUERY_CACHE_KEY:when_is_the_ntech_conference


'The NTECH internal engineering conference is set to take place from December 9-12, 2024. It will be a hybrid event with sessions in Santa Clara and streamed online. For more information, you can visit the NTECH website or join the conversation on the #ntech2024 Slack channel. Please let me know if you have any other questions or need further assistance.\n\nFor more details, please refer to the following links:\n- [NTECH 2024 to Run in December, Submissions Open June 17](https://nvidia.sharepoint.com/sites/NVINFO-Global-News/SitePages/NTECH-2024-to-Run-in-December,-Submissions-Open-June-17.aspx)\n- [Share Your Work: NTECH 2024 Submissions Window Closes July 19](https://nvidia.sharepoint.com/sites/NVINFO-Updates/SitePages/Share-Your-Work--NTECH-2024-Submissions-Window-Closes-July-19.aspx)'

In [6]:
key_list = cache.cache.hkeys("NVINFO_RESPONSE_CACHE")

In [8]:
key_list = [k for k in key_list if k.lower().startswith("nvinfo|")]
key_list

['NVINFO|can_we_bring_dogs_to_work',
 'NVINFO|where_can_i_find_information_on_employee_discounts',
 'NVINFO|do_we_have_access_to_linked_in_learning',
 'NVINFO|how_do_i_leave_a_d_l',
 'NVINFO|where_can_i_find_information_on_patents',
 'NVINFO|where_can_i_find_the_travel_expense_policy',
 'NVINFO|what_is_the_schedule_for_r_s_u_vest',
 'NVINFO|what_does_r_s_u_offset_mean_on_my_paycheck',
 'NVINFO|where_can_i_find_the_n_v_i_d_i_a_presentation_templates',
 'NVINFO|what_are_n_v_i_d_i_a_core_values',
 'NVINFO|how_do_i_join_a_d_l',
 'NVINFO|where_can_i_find_n_v_i_d_i_a_logos',
 'NVINFO|when_are_the_next_free_days',
 'NVINFO|what_is_the_employee_assistance_program',
 'NVINFO|how_do_i_report_phishing',
 'NVINFO|what_is_inspire_365',
 'NVINFO|what_is_my_alternate_i_d',
 'NVINFO|are_interns_eligible_for_e_s_p_p',
 'NVINFO|how_can_i_view_my_paycheck']

In [17]:
cache_answer = []
for key in key_list:
    cache_answer.append({"key": key, "answer": cache.get_from_cache(key, query_key_hash)})
# cache_answer

In [18]:
csv_file = "cached_answer_19_queries.csv"
import csv
# Writing to CSV
with open(csv_file, mode='w', newline='') as file:
    # Get the fieldnames from the keys of the first dictionary
    fieldnames = cache_answer[0].keys()
    
    # Create a writer object
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header
    writer.writeheader()
    
    # Write the data
    writer.writerows(cache_answer)

print(f"Data written to {csv_file}")

Data written to cached_answer_19_queries.csv


In [56]:
# ! pip3 install redisvl 0.33

In [22]:
! rvl stats

14:14:51 [RedisVL] ERROR   Index name or schema must be provided


In [62]:
from redisvl.extensions.llmcache import SemanticCache

import redis

redis_secret = {'password': 'vqWn4aQpHqTzm0ar',  'redis_url': 'clustercfg.redis-nonprd.orgtig.memorydb.us-west-2.amazonaws.com', 'username': 'nvbot-nonprod'}

query = "when_are_the_next_free_days"
query_key_hash = 'NVINFO_RESPONSE_CACHE'
query_key = f'NVINFO_QUERY_CACHE_KEY|{query}'


redis = redis.Redis(host=redis_secret["redis_url"], ssl=True, username=redis_secret["username"],
                                 password=redis_secret["password"], decode_responses=True)

info = redis.info()
# print ("info", info)
db_number = redis.connection_pool.connection_kwargs.get('db', 0)  # Default is 0 if not specified

# Extract and print the Redis version
redis_version = info.get("redis_version", "Unknown version")
print(f"Redis server version: {redis_version}, db_number: {db_number}")



Redis server version: 7.1.0, db_number: 0


In [ ]:
llmcache = SemanticCache(
    name="NVINFO_RESPONSE_CACHE",            # underlying search index name
    prefix="NVINFO",                   # redis key prefix for hash entries
    redis_client = redis,  # redis connection url string
    distance_threshold=0.1               # semantic cache distance threshold
)

In [ ]:
# # Function to insert data into the cache
# def insert_into_cache(key, value):
#     response = llmcache.insert(key=key, value=value)
#     print(f"Inserted into cache: {response}")
# 
# # Function to check if a key exists in the cache
# def check_cache(key):
#     cache_value = llmcache.get(key=key)
#     if cache_value:
#         print(f"Cache hit for key '{key}': {cache_value}")
#     else:
#         print(f"Cache miss for key '{key}'")

In [ ]:
# insert_key = "user_query_1"
# insert_value = {"response": "This is a cached response"}
# insert_into_cache(insert_key, insert_value)
# 
# # Check if the key exists in the cache
# check_key = "user_query_1"
# check_cache(check_key)